In [1]:
from datasets import Dataset
import pandas as pd
from determined.experimental import Determined
from determined.experimental import client
from determined import pytorch

# remove warnings
import warnings
warnings.filterwarnings('ignore')

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Set up .detignore file so the checkpoints directory is not packaged into future experiments
!echo checkpoints > .detignore

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
model_name = "tokyotech-llm/Swallow-13b-instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

## Inference test on the original HuggingFace model

In [4]:
device

device(type='cuda')

In [5]:
def answer_me(model, messages, device, is_peft=False, det_expid=None):
    
    # The model loaded from a Determined checkpoint is finetuned using PEFT
    if det_expid and is_peft:
        checkpoint = client.get_experiment(experiment_id).top_checkpoint(sort_by="eval_loss", smaller_is_better=True)
        path = checkpoint.download()
        model_path = path+'/checkpoint-100' # Replace 100 with the length total number of batches
        model = PeftModel.from_pretrained(model, model_path, torch_dtype=torch.bfloat16, device_map="auto")
        
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

    model_inputs = encodeds.to(device)
    model.to(device)

    generated_ids = model.generate(model_inputs, max_new_tokens=128, do_sample=True)
    decoded = tokenizer.batch_decode(generated_ids)
    print(decoded[0])

In [6]:
messages = [
    {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
    {"role": "user", "content": "東京工業大学の主なキャンパスについて教えてください"}
]

answer_me(model, messages, device, is_peft=False, det_expid=None)

tokenizer_config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/457 [00:00<?, ?B/s]

<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

東京工業大学の主なキャンパスについて教えてください [/INST] 東京工業大学は、日本の東京都にある国立大学で、主に4つのキャンパスを持っています。

1. 大岡山キャンパス (O-Okayama Campus)

大岡山キャンパスは、東京工業大学のメインキャンパスで、東京都目黒区にあります。ここには、大学の研究所や学部、大学院などがあり、多くの学生や研究者が集まっています。

2. すずかけ台キャンパス (Suzukakedai Campus)

すずかけ台キャンパスは、東京都町田市にあり


## Inference test on the original HuggingFace model for a math question

In [7]:
messages = [
    {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
    {"role": "user", "content": "What is the value of 'y' when x = 3 in the equation y = 2x + 1?"}
]

answer_me(model, messages, device, is_peft=False, det_expid=None)

<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

What is the value of 'y' when x = 3 in the equation y = 2x + 1? [/INST]  The value of y when x = 3 is 2x + 1. 

So, y = 2*3 + 1 = 6 + 1 = 7.</s>


## Inference test on the finetuned model

In [8]:
experiment_id = 12218
messages = [
        {"role": "system", "content": "あなたは誠実で優秀な日本人のアシスタントです。"},
        {"role": "user", "content": "What is the value of 'y' when x = 3 in the equation y = 2x + 1?."}
    ]
answer_me(model, messages, device, is_peft=True, det_expid=experiment_id)

<s> [INST] <<SYS>>
あなたは誠実で優秀な日本人のアシスタントです。
<</SYS>>

What is the value of 'y' when x = 3 in the equation y = 2x + 1?. [/INST]  The value of y when x = 3 is 7.</s>
